Imports

In [2]:
import obspy
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from obspy import read, read_inventory, Stream
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader
from obspy.geodetics import locations2degrees
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

Downloading data

In [3]:
def download_ground_motion_data(event_time, latitude, longitude):
    """
    Downloads seismic waveform data along with instrument response metadata.
    """
    domain = CircularDomain(
        latitude=latitude,
        longitude=longitude,
        minradius=0.0,
        maxradius=20.0
    )
    
    restrictions = Restrictions(
        starttime=event_time - 60,
        endtime=event_time + 600,
        reject_channels_with_gaps=True,
        minimum_length=0.90,
        minimum_interstation_distance_in_m=20000,
        channel_priorities=["HN[ZNE]", "BN[ZNE]", "HH[ZNE]", "BH[ZNE]"],
        location_priorities=["", "00", "10"]
    )
    
    mdl = MassDownloader()
    mdl.download(
        domain,
        restrictions,
        mseed_storage="waveforms_motion",
        stationxml_storage="stations_motion"
    )
    
    print("Seismic data and instrument response downloaded successfully!")

event_time = obspy.UTCDateTime(2011, 3, 11, 5, 47, 32)
event_latitude = 37.52
event_longitude = 143.04

download_ground_motion_data(event_time, event_latitude, event_longitude)


[2025-03-26 23:00:01,082] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for AUSPASS, BGR, EIDA, EMSC, ETH, GEOFON, GEONET, GFZ, ICGC, IESDMC, INGV, IPGP, ISC, KNMI, KOERI, LMU, NCEDC, NIEP, NOA, RESIF, RESIFPH5, SCEDC, TEXNET, UIB-NORSAR, USGS, USP, ORFEUS, IRIS.
[2025-03-26 23:00:04,421] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'ISC' as it does not have 'dataselect' and/or 'station' services.
[2025-03-26 23:00:04,656] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'EMSC' as it does not have 'dataselect' and/or 'station' services.
[2025-03-26 23:00:04,857] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'RESIFPH5' as it does not have 'dataselect' and/or 'station' services.
[2025-03-26 23:00:04,877] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'USGS' as it does not have 'dataselect' and/or 'station' services.
[2025-03-26 23:00:17,591] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use 

Seismic data and instrument response downloaded successfully!


In [ ]:
for file in glob.glob("waveforms_motion/*.mseed"):
    st = read(file)
    st.plot()
    break

Process the downloaded data

In [ ]:
def process_ground_motion_data():
    """
    Reads downloaded waveforms, removes instrument response to get acceleration data,
    and saves corrected waveforms.
    """
    st = Stream()
    
    for file in glob.glob("waveforms_motion/*.mseed"):
        st += read(file)
    
    inv = read_inventory("stations_motion/*")
    
    st_acc = Stream()
    
    for tr in st:
        try:
            tr_acc = tr.copy()
            tr_acc.remove_response(inventory=inv, output="ACC")
            st_acc += tr_acc
        except Exception as e:
            print(f"Error processing trace {tr.id}: {e}")
    
    st_acc.write("waveforms_acceleration.mseed", format="MSEED")
    print("Instrument response removed, acceleration data saved as 'waveforms_acceleration.mseed'")
    
    return st, st_acc

st_raw, st_acc = process_ground_motion_data()


Feature extraction

In [ ]:
def extract_features_for_prediction(event_latitude, event_longitude, st_acc):
    """
    Extract comprehensive features from processed acceleration waveforms for prediction model.
    """
    data_list = []
    
    for tr in st_acc:
        station_lat, station_lon, station_elev = None, None, None
        
        if hasattr(tr.stats, 'sac'):
            station_lat = tr.stats.sac.stla if hasattr(tr.stats.sac, 'stla') else None
            station_lon = tr.stats.sac.stlo if hasattr(tr.stats.sac, 'stlo') else None
            station_elev = tr.stats.sac.stel if hasattr(tr.stats.sac, 'stel') else None
        
        if station_lat is None or station_lon is None:
            try:
                inv = read_inventory(f"stations_motion/{tr.stats.network}.{tr.stats.station}.xml")
                coords = inv.get_coordinates(tr.id)
                station_lat = coords['latitude']
                station_lon = coords['longitude']
                station_elev = coords['elevation']
            except Exception as e:
                print(f"Could not get coordinates for {tr.id}: {e}")
                continue
        
        distance_to_event = locations2degrees(station_lat, station_lon, event_latitude, event_longitude)
        
        pga = np.max(np.abs(tr.data))
        rms_acc = np.sqrt(np.mean(np.square(tr.data)))
        arias_intensity = np.trapz(np.square(tr.data), dx=tr.stats.delta) * np.pi / (2 * 9.81)
        
        cumulative_energy = np.cumsum(np.square(tr.data))
        sig_duration = 0
        if cumulative_energy[-1] > 0:
            normalized_energy = cumulative_energy / cumulative_energy[-1]
            idx_5 = np.where(normalized_energy >= 0.05)[0][0] if len(np.where(normalized_energy >= 0.05)[0]) > 0 else 0
            idx_95 = np.where(normalized_energy >= 0.95)[0][0] if len(np.where(normalized_energy >= 0.95)[0]) > 0 else len(normalized_energy)-1
            sig_duration = (idx_95 - idx_5) * tr.stats.delta
        
        try:
            from scipy import signal
            frequencies, psd = signal.welch(tr.data, fs=1/tr.stats.delta, nperseg=min(4096, len(tr.data)))
            predominant_freq = frequencies[np.argmax(psd)]
            mean_freq = np.sum(frequencies * psd) / np.sum(psd) if np.sum(psd) > 0 else 0
            spectral_centroid = mean_freq
            
            low_freq_energy = np.sum(psd[(frequencies >= 0) & (frequencies <= 1)]) if np.any((frequencies >= 0) & (frequencies <= 1)) else 0
            mid_freq_energy = np.sum(psd[(frequencies > 1) & (frequencies <= 5)]) if np.any((frequencies > 1) & (frequencies <= 5)) else 0
            high_freq_energy = np.sum(psd[frequencies > 5]) if np.any(frequencies > 5) else 0
        except Exception as e:
            print(f"Error in frequency analysis for {tr.id}: {e}")
            predominant_freq = mean_freq = spectral_centroid = low_freq_energy = mid_freq_energy = high_freq_energy = 0
        
        data_list.append({
            'station_id': f"{tr.stats.network}.{tr.stats.station}.{tr.stats.location}.{tr.stats.channel}",
            'station_lat': station_lat,
            'station_lon': station_lon,
            'station_elev': station_elev,
            'distance_to_event': distance_to_event,
            'pga': pga,
            'rms_acc': rms_acc,
            'arias_intensity': arias_intensity,
            'sig_duration': sig_duration,
            'predominant_freq': predominant_freq,
            'mean_freq': mean_freq,
            'spectral_centroid': spectral_centroid,
            'low_freq_energy': low_freq_energy,
            'mid_freq_energy': mid_freq_energy,
            'high_freq_energy': high_freq_energy,
            'component': tr.stats.channel[-1]
        })
    
    features_df = pd.DataFrame(data_list)
    features_df.to_csv("extracted_acceleration_features.csv", index=False)
    print("Acceleration features extracted and saved to 'extracted_acceleration_features.csv'")
    
    return features_df


features_df = extract_features_for_prediction(event_latitude, event_longitude, st_acc)


KNN

In [ ]:
def train_knn_model(features_df):
    """
    Train a kNN model to predict ground acceleration at new locations.
    """
    X = features_df[['distance_to_event', 'station_lat', 'station_lon', 'station_elev',
                     'predominant_freq', 'mean_freq']].values
    y = features_df['pga'].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    knn = KNeighborsRegressor(n_neighbors=5, weights='distance')
    knn.fit(X_train, y_train)
    
    y_pred = knn.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Model Performance - MSE: {mse:.4f}, R²: {r2:.4f}")
    
    return knn, X_test, y_test, y_pred

knn, X_test, y_test, y_pred = train_knn_model(features_df)


Visualization

In [ ]:
def visualize_results(features_df, X_test, y_test, y_pred):
    """
    Visualize the acceleration prediction results.
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.7)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
    plt.xlabel("Actual PGA")
    plt.ylabel("Predicted PGA")
    plt.title("Ground Acceleration: Actual vs Predicted")
    plt.savefig("actual_vs_predicted_pga.png", dpi=300)
    plt.close()
    
    plt.figure(figsize=(10, 6))
    plt.scatter(features_df['distance_to_event'], features_df['pga'], alpha=0.7)
    plt.xlabel("Distance to Epicenter (degrees)")
    plt.ylabel("Peak Ground Acceleration (PGA)")
    plt.yscale('log')
    plt.title("PGA vs Distance")
    plt.savefig("pga_vs_distance.png", dpi=300)
    plt.close()
    
    if len(features_df) >= 10:
        try:
            from mpl_toolkits.basemap import Basemap
            
            plt.figure(figsize=(12, 8))
            event_lat, event_lon = 37.52, 143.04
            m = Basemap(projection='merc', llcrnrlat=event_lat-20, urcrnrlat=event_lat+20,
                        llcrnrlon=event_lon-20, urcrnrlon=event_lon+20, resolution='i')
            
            m.drawcoastlines()
            m.drawcountries()
            m.drawmapboundary(fill_color='aqua')
            m.fillcontinents(color='coral', lake_color='aqua')
            
            x, y = m(features_df['station_lon'].values, features_df['station_lat'].values)
            scatter = m.scatter(x, y, c=features_df['pga'], cmap='viridis', s=100, marker='o', 
                                edgecolor='k', alpha=0.7)
            
            ex, ey = m(event_lon, event_lat)
            m.plot(ex, ey, 'r*', markersize=20)
            
            plt.colorbar(scatter, label='Peak Ground Acceleration')
            plt.title('PGA Distribution Map')
            plt.savefig("pga_map.png", dpi=300)
            plt.close()
        except ImportError:
            print("Basemap not available. Skipping map visualization.")

visualize_results(features_df, X_test, y_test, y_pred)
